In [38]:
import numpy as np
import cv2

# 读入图像并二值化
img = cv2.imread('assets/star1.png', 0)
img_color = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
ret,thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# 寻找轮廓，图像中第一个轮廓就是需要的轮廓，
# 如果在其他程序中，需要根据一定条件进行筛选
# 图像矩可以计算一些特征，例如：物体的质心，物体的面积
# Cx = M10 / M00, Cy = M01 / M00
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnt = contours[0]
M = cv2.moments(cnt)
print("星号的矩为:", M )
img_color0 = img_color.copy()
cv2.drawContours(img_color0, [cnt], 0, (0, 128, 255), 3, maxLevel=0)
cv2.imshow("contours-src", img_color0)


# contour的面积可以直接使用 contourArea
# 也可以用 M['m00'] 来获得
area = cv2.contourArea(cnt)
print("星号的面积为：", area)

# 输出周长
# 第二个参数如果为True，则为闭合的轮廓，否则为弧
perimeter = cv2.arcLength(cnt, True)
print("星号的周长为：", perimeter)

# 轮廓近似
# 用另一个更少点的形状去拟合原来的轮廓（使用算法为 道格拉斯-普克算法 ， 
# 将曲线近似表示为一系列点，并减少点的数量的一种 算法）
img_color1 = img_color.copy()
epsilon = 0.1 * perimeter
approx = cv2.approxPolyDP(cnt, epsilon, True)
cv2.drawContours(img_color1, [approx], 0, (0, 255, 0), 3, maxLevel=0)
cv2.imshow("0.1-approxPolyDP", img_color1)

# 拟合精度更高，需要的点回更多一些
img_color2 = img_color.copy()
epsilon2 = 0.005 * perimeter
approx2 = cv2.approxPolyDP(cnt, epsilon2, True)
cv2.drawContours(img_color2, [approx2], 0, (0, 255, 255), 3, maxLevel=0)
cv2.imshow("0.005-approxPolyDP", img_color2)

cv2.waitKey(0)
cv2.destroyAllWindows()


星号的矩为: {'m00': 130522.0, 'm10': 33632144.0, 'm01': 24005228.166666664, 'm20': 10974770282.666666, 'm11': 6224358011.416666, 'm02': 5406270574.5, 'm30': 4011914771313.0, 'm21': 2046283287276.75, 'm12': 1408108180508.5833, 'm03': 1363179635382.75, 'mu20': 2308636527.15621, 'mu11': 38831506.65739727, 'mu02': 991298544.2389975, 'mu30': -5751574310.887695, 'mu21': 7823576580.904053, 'mu12': 768507355.5574036, 'mu03': 4240497589.183838, 'nu20': 0.13551523513082395, 'nu11': 0.0022793803586064827, 'nu02': 0.058188482130994895, 'nu30': -0.0009344959758346628, 'nu21': 0.001271147765169143, 'nu12': 0.00012486442708533664, 'nu03': 0.000688981436808956}
星号的面积为： 130522.0
星号的周长为： 2386.9503506422043


In [29]:
# 什么是凸包？
# 凸: 凸对象内部任意两点连接所有点都在图像内部，则为凸对象，否则为非凸或凹对象。
# 包: 能够完全包住对象的最小形状

import numpy as np
import cv2

# 读入灰度图像，并阈值化
image = cv2.imread("worldmap.png", 0)
ret, thresh = cv2.threshold(image, 200, 255, cv2.THRESH_BINARY_INV)

# 寻找阈值图像的轮廓
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

# # 画出所有轮廓
map_color = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
cv2.drawContours(map_color, contours, -1, (0, 255, 0), 1)

# 计算各个轮廓的凸包
hull = []
for i in range(len(contours)):
    hull.append(cv2.convexHull(contours[i], False))

# 创建一个黑色的画布
drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)
# 画轮廓和凸包点
for i in range(len(contours)):
    color_contours = (0, 255, 0)  # green - color for contours
    hullcolor = (0, 0, 255)  # blue - color for convex hull
    
    # 画轮廓
    cv2.drawContours(drawing, contours, i, color_contours, 1, 8, hierarchy)
    
    # 画凸包
    cv2.drawContours(drawing, hull, i, hullcolor, 1, 8)

cv2.imshow("drawing", drawing)
cv2.imshow("map_color", map_color)
cv2.imshow("thresh", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [2]:
import numpy as np
import cv2

# 读入灰度图像，并阈值化
image = cv2.imread("testfit.png", 0)
ret, thresh = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY)
colorim = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

# 寻找阈值图像的轮廓
contours, hierarchy = cv2.findContours(
    thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

cnt = contours[0]

# 找到最小的外接矩形
boundingRectImg = colorim.copy()
x,y,w,h = cv2.boundingRect(cnt)
cv2.rectangle(boundingRectImg, (x,y), (x+w, y+h), (0, 255, 0), 2)

# 找到最小倾斜矩形
minAreaRectImg = colorim.copy()
rect = cv2.minAreaRect(cnt)
box = cv2.boxPoints(rect)
box = np.int0(box)
cv2.drawContours(minAreaRectImg, [box], 0, (0, 0, 255), 2)

# 最小圆拟合
minEnclosingCircleImg = colorim.copy()
(x,y),radius = cv2.minEnclosingCircle(cnt)
center = (int(x),int(y))
radius = int(radius)
cv2.circle(minEnclosingCircleImg, center, radius, (0,255,0), 2)

# 最小椭圆拟合
fitEllipseImg = colorim.copy()
ellipse = cv2.fitEllipse(cnt)
cv2.ellipse(fitEllipseImg, ellipse, (0,255,0), 2)

# 直线拟合
fitLineImg = colorim.copy()
rows, cols = image.shape[:2]
[vx, vy, x, y] = cv2.fitLine(cnt, cv2.DIST_L2, 0, 0.01, 0.01)
lefty = int((-x*vy/vx) + y)
righty = int(((cols-x)*vy/vx)+y)
cv2.line(fitLineImg, (cols-1, righty), (0, lefty), (0, 255, 0), 2)


cv2.imshow("thresh", thresh)
cv2.imshow("boundingRect", boundingRectImg)
cv2.imshow("minAreaRect", minAreaRectImg)
cv2.imshow("minEnclosingCircleImg", minEnclosingCircleImg)
cv2.imshow("fitEllipseImg", fitEllipseImg)
cv2.imshow("fitLineImg", fitLineImg)

cv2.waitKey(0)
cv2.destroyAllWindows()